In [1]:
%run ../spark-default.py

In [2]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")

In [3]:
import builtins as b
from datetime import datetime, timedelta, date
import sys, traceback
from delta.tables import DeltaTable

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# spark.sql("optimize stage.ciha_step1")

In [5]:
# Até aqui, a base do CIHA foi enriquecida com:
#  - Nome das cidades (residencia e movimentação)
#  - Nome, uf e info dos estados (residencia e movimentação)
#  - Infos do IBGE como população e área
#  - Calculo de feriados

In [4]:
ciha_step2 = spark.table("stage.ciha_step2")
ciha_step2.printSchema()

root
 |-- mes_cmpt: string (nullable = true)
 |-- espec: string (nullable = true)
 |-- cgc_hosp: string (nullable = true)
 |-- munic_res: string (nullable = true)
 |-- nasc: date (nullable = true)
 |-- sexo: string (nullable = true)
 |-- uti_mes_to: string (nullable = true)
 |-- uti_int_to: string (nullable = true)
 |-- proc_rea: string (nullable = true)
 |-- qt_proc: string (nullable = true)
 |-- dt_atend: date (nullable = true)
 |-- dt_atend_dow: integer (nullable = true)
 |-- dt_saida: date (nullable = true)
 |-- diag_princ: string (nullable = true)
 |-- diag_secun: string (nullable = true)
 |-- cobranca: string (nullable = true)
 |-- natureza: string (nullable = true)
 |-- gestao: string (nullable = true)
 |-- munic_mov: string (nullable = true)
 |-- cod_idade: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- dias_perm: string (nullable = true)
 |-- morte: string (nullable = true)
 |-- nacional: string (nullable = true)
 |-- car_int: string (nullable = true)
 |-- 

In [5]:
idhm = spark.table("stage.idhm")
idhm

cod_municipio,idhm_2010
110001,0.641
110002,0.702
110003,0.65
110004,0.718
110005,0.692
110006,0.685
110007,0.613
110008,0.611
110009,0.672
110010,0.657


In [6]:
df_procedimentos = spark.table("stage.procedimentos")
df_procedimentos

co_procedimento,no_procedimento,tp_complexidade,tp_sexo,qt_maxima_execucao,qt_dias_permanencia,qt_pontos,vl_idade_minima,vl_idade_maxima,vl_sh,vl_sa,vl_sp,co_financiamento,co_rubrica,qt_tempo_permanencia,ds_procedimento,no_grupo,no_sub_grupo,no_forma_organizacao
0101010010,ATIVIDADE EDUCATI...,1,N,9999,9999,0000,9999,9999,0000000000,0000000000,0000000000,01,,9999,CONSISTE NAS ATIV...,Ações de promoção...,Ações coletivas/i...,Educação em saúde
0101010028,ATIVIDADE EDUCATI...,2,I,9999,9999,0000,0084,1571,0000000000,0000000270,0000000000,06,,9999,CONSISTE NAS ATIV...,Ações de promoção...,Ações coletivas/i...,Educação em saúde
0101010036,PRÁTICA CORPORAL ...,1,I,9999,9999,0000,0072,1571,0000000000,0000000000,0000000000,01,,9999,ATIVIDADE FÍSICA ...,Ações de promoção...,Ações coletivas/i...,Educação em saúde
0101010095,PREVENÇÃO DA COVI...,1,I,9999,9999,0000,0000,1571,0000000000,0000000000,0000000000,01,,9999,AÇÕES COLETIVAS D...,Ações de promoção...,Ações coletivas/i...,Educação em saúde
0101010109,ATIVIDADES EDUCAT...,1,I,9999,9999,0000,0000,1571,0000000000,0000000000,0000000000,07,,9999,PROMOVER OU REALI...,Ações de promoção...,Ações coletivas/i...,Educação em saúde
0101010117,ATIVIDADE DE EDUC...,2,N,9999,9999,0000,9999,9999,0000000000,0000000000,0000000000,05,,9999,ENTENDE-SE A EDUC...,Ações de promoção...,Ações coletivas/i...,Educação em saúde
0101020015,AÇÃO COLETIVA DE ...,1,I,9999,9999,0000,0048,1571,0000000000,0000000000,0000000000,01,,9999,APLICAÇÃO TÓPICA ...,Ações de promoção...,Ações coletivas/i...,Educação em saúde
0101020023,AÇÃO COLETIVA DE ...,1,I,9999,9999,0000,0084,1571,0000000000,0000000000,0000000000,01,,9999,BOCHECHO DE SOLUÇ...,Ações de promoção...,Ações coletivas/i...,Educação em saúde
0101020031,AÇÃO COLETIVA DE ...,1,I,9999,9999,0000,0036,1571,0000000000,0000000000,0000000000,01,,9999,ESCOVAÇÃO DENTAL ...,Ações de promoção...,Ações coletivas/i...,Educação em saúde
0101020040,AÇÃO COLETIVA DE ...,1,I,9999,9999,0000,0000,1571,0000000000,0000000000,0000000000,01,,9999,COMPREENDE A AVAL...,Ações de promoção...,Ações coletivas/i...,Educação em saúde


In [7]:
ciha_step3 = ciha_step2.alias("ciha") \
    .join(
        idhm.alias("idhm1"),
        col("ciha.munic_res") == col("idhm1.cod_municipio"),
        "left"
    ) \
    .join(
        idhm.alias("idhm2"),
        col("ciha.munic_mov") == col("idhm2.cod_municipio"),
        "left"
    ) \
    .join(
        df_procedimentos.alias("proc"),
        col("ciha.proc_rea") == col("proc.co_procedimento"),
        "left"
    ) \
    .select(
        col("ciha.mes_cmpt"),
        col("ciha.espec"),
        col("ciha.cgc_hosp"),
        col("ciha.munic_res"),
        col("ciha.nasc"),
        col("ciha.sexo"),
        col("ciha.uti_mes_to"),
        col("ciha.uti_int_to"),
        col("ciha.proc_rea"),
        col("ciha.qt_proc"),
        col("ciha.dt_atend"),
        col("ciha.dt_atend_dow"),
        col("ciha.dt_saida"),
        col("ciha.diag_princ"),
        col("ciha.diag_secun"),
        col("ciha.cobranca"),
        col("ciha.natureza"),
        col("ciha.gestao"),
        col("ciha.munic_mov"),
        col("ciha.cod_idade"),
        col("ciha.idade"),
        col("ciha.dias_perm"),
        col("ciha.morte"),
        col("ciha.nacional"),
        col("ciha.car_int"),
        col("ciha.homonimo"),
        col("ciha.cnes"),
        col("ciha.fonte"),
        col("ciha.cgc_consor"),
        col("ciha.modalidade"),
        col("ciha.input_file_name"),
        col("ciha.arquivo_sigla_uf"),
        col("ciha.ano_cmpt"),
        col("ciha.munic_res_nome_uf"),
        col("ciha.munic_mov_nome_uf"),
        col("ciha.munic_res_nome_municipio"),
        col("ciha.munic_mov_nome_municipio"),
        col("ciha.munic_res_sigla_uf"),
        col("ciha.munic_mov_sigla_uf"),
        col("ciha.munic_res_regiao"),
        col("ciha.munic_mov_regiao"),
        col("ciha.munic_res_populacao_residente"),
        col("ciha.munic_mov_populacao_residente"),
        col("ciha.munic_res_area_unidade_territorial"),
        col("ciha.munic_mov_area_unidade_territorial"),
        
        col("ciha.feriado.feriado"),
        col("ciha.feriado.distancia_feriado").alias("feriado_distancia"),
        col("ciha.feriado.feriado_info"),

        col("ciha.diag_princ_cid10.sub_categoria").alias("diag_princ_sub_categoria"),
        col("ciha.diag_princ_cid10.sub_categoria_classificacao").alias("diag_princ_sub_categoria_classificacao"),
        col("ciha.diag_princ_cid10.restringido_sexo").alias("diag_princ_restringido_sexo"),
        col("ciha.diag_princ_cid10.causa_obito").alias("diag_princ_causa_obito"),
        col("ciha.diag_princ_cid10.sub_categoria_descricao").alias("diag_princ_sub_categoria_descricao"),
        col("ciha.diag_princ_cid10.sub_categoria_descricao_abreviada").alias("diag_princ_sub_categoria_descricao_abreviada"),
        col("ciha.diag_princ_cid10.sub_categoria_referencia").alias("diag_princ_sub_categoria_referencia"),
        col("ciha.diag_princ_cid10.sub_categoria_excluidos").alias("diag_princ_sub_categoria_excluidos"),
        col("ciha.diag_princ_cid10.categoria").alias("diag_princ_categoria"),
        col("ciha.diag_princ_cid10.categoria_classificacao").alias("diag_princ_categoria_classificacao"),
        col("ciha.diag_princ_cid10.categoria_descricao").alias("diag_princ_categoria_descricao"),
        col("ciha.diag_princ_cid10.categoria_descricao_abreviada").alias("diag_princ_categoria_descricao_abreviada"),
        col("ciha.diag_princ_cid10.categoria_referencia").alias("diag_princ_categoria_referencia"),
        col("ciha.diag_princ_cid10.capitulo_numero").alias("diag_princ_capitulo_numero"),
        col("ciha.diag_princ_cid10.capitulo_descricao").alias("diag_princ_capitulo_descricao"),
        col("ciha.diag_princ_cid10.capitulo_descricao_abreviada").alias("diag_princ_capitulo_descricao_abreviada"),

        col("ciha.diag_secun_cid10.sub_categoria").alias("diag_secun_sub_categoria"),
        col("ciha.diag_secun_cid10.sub_categoria_classificacao").alias("diag_secun_sub_categoria_classificacao"),
        col("ciha.diag_secun_cid10.restringido_sexo").alias("diag_secun_restringido_sexo"),
        col("ciha.diag_secun_cid10.causa_obito").alias("diag_secun_causa_obito"),
        col("ciha.diag_secun_cid10.sub_categoria_descricao").alias("diag_secun_sub_categoria_descricao"),
        col("ciha.diag_secun_cid10.sub_categoria_descricao_abreviada").alias("diag_secun_sub_categoria_descricao_abreviada"),
        col("ciha.diag_secun_cid10.sub_categoria_referencia").alias("diag_secun_sub_categoria_referencia"),
        col("ciha.diag_secun_cid10.sub_categoria_excluidos").alias("diag_secun_sub_categoria_excluidos"),
        col("ciha.diag_secun_cid10.categoria").alias("diag_secun_categoria"),
        col("ciha.diag_secun_cid10.categoria_classificacao").alias("diag_secun_categoria_classificacao"),
        col("ciha.diag_secun_cid10.categoria_descricao").alias("diag_secun_categoria_descricao"),
        col("ciha.diag_secun_cid10.categoria_descricao_abreviada").alias("diag_secun_categoria_descricao_abreviada"),
        col("ciha.diag_secun_cid10.categoria_referencia").alias("diag_secun_categoria_referencia"),
        col("ciha.diag_secun_cid10.capitulo_numero").alias("diag_secun_capitulo_numero"),
        col("ciha.diag_secun_cid10.capitulo_descricao").alias("diag_secun_capitulo_descricao"),
        col("ciha.diag_secun_cid10.capitulo_descricao_abreviada").alias("diag_secun_capitulo_descricao_abreviada"),

        col("idhm1.idhm_2010").alias("munic_res_idhm"),
        col("idhm2.idhm_2010").alias("munic_mov_idhm"),

        col("proc.no_procedimento").alias("proc_nome"),
        col("proc.tp_complexidade").alias("proc_tp_complexidade"),
        col("proc.tp_sexo").alias("proc_tp_sexo"),
        col("proc.qt_maxima_execucao").alias("proc_qt_max_exec"),
        col("proc.qt_dias_permanencia").alias("proc_qt_dias_perm"),
        col("proc.qt_pontos").alias("proc_qt_pontos"),
        col("proc.ds_procedimento").alias("proc_ds_procedimento"),
        col("proc.no_grupo").alias("proc_no_grupo"),
        col("proc.no_sub_grupo").alias("proc_no_sub_grupo"),
        col("proc.no_forma_organizacao").alias("proc_no_forma_organizacao")
    )

    
ciha_step3.printSchema()
ciha_step3.show(truncate=False)

root
 |-- mes_cmpt: string (nullable = true)
 |-- espec: string (nullable = true)
 |-- cgc_hosp: string (nullable = true)
 |-- munic_res: string (nullable = true)
 |-- nasc: date (nullable = true)
 |-- sexo: string (nullable = true)
 |-- uti_mes_to: string (nullable = true)
 |-- uti_int_to: string (nullable = true)
 |-- proc_rea: string (nullable = true)
 |-- qt_proc: string (nullable = true)
 |-- dt_atend: date (nullable = true)
 |-- dt_atend_dow: integer (nullable = true)
 |-- dt_saida: date (nullable = true)
 |-- diag_princ: string (nullable = true)
 |-- diag_secun: string (nullable = true)
 |-- cobranca: string (nullable = true)
 |-- natureza: string (nullable = true)
 |-- gestao: string (nullable = true)
 |-- munic_mov: string (nullable = true)
 |-- cod_idade: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- dias_perm: string (nullable = true)
 |-- morte: string (nullable = true)
 |-- nacional: string (nullable = true)
 |-- car_int: string (nullable = true)
 |-- 

In [12]:
ciha_step3.filter("proc_nome is null").select("proc_rea").distinct().show(truncate=False)

+----------+
|proc_rea  |
+----------+
|0202120007|
|0309030005|
|0412040000|
|0212010000|
|0214010007|
|0413010007|
|0204010187|
|0406030006|
|0309040000|
|0202080005|
|0303170000|
|0306020009|
|0407030000|
|0501020004|
|0101030002|
|0211090000|
|0304060003|
|0211010006|
|0506010007|
|0211110000|
+----------+
only showing top 20 rows



In [8]:
ciha_nulls = ciha_step3.select(count("*").alias("total"), *[count(when(col(c).isNull(), c)).alias(c) for c in ciha_step3.columns]).first().asDict()

ciha_nulls_total = ciha_nulls["total"]

df_ciha_nulls = spark.createDataFrame(
    [{"col": k, "count": v, "percentual": b.round((v/ciha_nulls_total) * 100, 2)} for k,v in ciha_nulls.items()]
) #.orderBy(col("count").desc())

df_ciha_nulls.show(200, truncate=False)

+--------------------------------------------+---------+----------+
|col                                         |count    |percentual|
+--------------------------------------------+---------+----------+
|total                                       |126778574|100.0     |
|mes_cmpt                                    |0        |0.0       |
|espec                                       |126778574|100.0     |
|cgc_hosp                                    |4175316  |3.29      |
|munic_res                                   |0        |0.0       |
|nasc                                        |0        |0.0       |
|sexo                                        |0        |0.0       |
|uti_mes_to                                  |0        |0.0       |
|uti_int_to                                  |1137983  |0.9       |
|proc_rea                                    |0        |0.0       |
|qt_proc                                     |0        |0.0       |
|dt_atend                                    |0 

In [13]:
# spark.sql("drop table stage.ciha_step3")

""


In [9]:
spark.sql("delete from stage.ciha_step3")

num_affected_rows
117092917


In [14]:
DeltaTable.createIfNotExists(spark) \
  .tableName("stage.ciha_step3") \
  .addColumns(ciha_step3.schema) \
  .partitionedBy("ano_cmpt") \
  .execute()

In [10]:
ciha_step3.write.mode("overwrite").insertInto("stage.ciha_step3")

In [11]:
spark.sql("optimize stage.ciha_step3")

path,metrics
s3a://datalake/st...,"{14, 33, {2187293..."


In [12]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")

In [13]:
spark.sql("VACUUM stage.ciha_step3 RETAIN 0 HOURS")

path
s3a://datalake/st...


In [14]:
spark.stop()